In [1]:
#Inspirasjon / kilder = https://stackabuse.com/linear-regression-in-python-with-scikit-learn/
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/Users/eliasnilsen/Desktop/Studie/VÅR2021/INFO284/Exam/nyc-rolling-sales.csv')

In [3]:
data.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [4]:
#Sletter ubrukelig data
data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop('EASE-MENT', axis=1, inplace=True)

In [5]:
#Sjekker for duplicates i datasettet
sum(data.duplicated())

765

In [6]:
#Sletter duplicates og sjekker om det funket
data = data.drop_duplicates(data.columns, keep='last')
sum(data.duplicated())

0

In [7]:
#Noen av feltene inneholder et - tegn, vi bytter ut disse med 0
#' -  ' finnes i en eller flere celler, vi må fjerne denne for å unngå errors
data['GROSS SQUARE FEET'] = data['GROSS SQUARE FEET'].str.strip()
data['SALE PRICE'] = data['SALE PRICE'].str.strip()
data.replace({"-":0},inplace=True)
data.replace(' ',np.nan, inplace=True)
#Tomt felt som ikke er satt opp riktig
data.replace(' -  ', np.nan)
df1=data.copy()

In [8]:
#Sjekker datatypen i settet
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83783 entries, 0 to 84547
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   BOROUGH                         83783 non-null  int64 
 1   NEIGHBORHOOD                    83783 non-null  object
 2   BUILDING CLASS CATEGORY         83783 non-null  object
 3   TAX CLASS AT PRESENT            83045 non-null  object
 4   BLOCK                           83783 non-null  int64 
 5   LOT                             83783 non-null  int64 
 6   BUILDING CLASS AT PRESENT       83045 non-null  object
 7   ADDRESS                         83783 non-null  object
 8   APARTMENT NUMBER                18849 non-null  object
 9   ZIP CODE                        83783 non-null  int64 
 10  RESIDENTIAL UNITS               83783 non-null  int64 
 11  COMMERCIAL UNITS                83783 non-null  int64 
 12  TOTAL UNITS                     83783 non-null

In [9]:
df1.dropna()
df1[['GROSS SQUARE FEET', 'SALE PRICE']] = df1[['GROSS SQUARE FEET', 'SALE PRICE']].astype(int)
df1.info()
df1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83783 entries, 0 to 84547
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   BOROUGH                         83783 non-null  int64 
 1   NEIGHBORHOOD                    83783 non-null  object
 2   BUILDING CLASS CATEGORY         83783 non-null  object
 3   TAX CLASS AT PRESENT            83045 non-null  object
 4   BLOCK                           83783 non-null  int64 
 5   LOT                             83783 non-null  int64 
 6   BUILDING CLASS AT PRESENT       83045 non-null  object
 7   ADDRESS                         83783 non-null  object
 8   APARTMENT NUMBER                18849 non-null  object
 9   ZIP CODE                        83783 non-null  int64 
 10  RESIDENTIAL UNITS               83783 non-null  int64 
 11  COMMERCIAL UNITS                83783 non-null  int64 
 12  TOTAL UNITS                     83783 non-null

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,NaN,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,C7,234 EAST 4TH STREET,NaN,10009,28,3,31,4616,18690,1900,2,C7,0,2016-12-14 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,C7,197 EAST 3RD STREET,NaN,10009,16,1,17,2212,7803,1900,2,C7,0,2016-12-09 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,NaN,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,NaN,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [10]:
#Vi vil bruke dataen fra GROSS SQUARE FEET kollonen til å forutse SALE PRICE
X = df1['GROSS SQUARE FEET'].values.reshape(-1, 1)
y = df1['SALE PRICE'].values.reshape(-1, 1)

In [11]:
#Deler opp dataen til trening og testing av modellen
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
#Trener dataen
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [15]:
#Intercept og slope
print(X.shape)
print(y.shape)
print("regressor.coef_: {}".format(regressor.coef_))
print("regressor.intercept_: {}".format(regressor.intercept_))

(83783, 1)
(83783, 1)
regressor.coef_: [[89.73817544]]
regressor.intercept_: [803650.95824498]


In [14]:
print("Training set score:  {:.2f}".format(regressor.score(X_train,y_train)))
print("Test set score:  {:.2f}".format(regressor.score(X_train,y_test)))

Training set score:  0.07


ValueError: Found input variables with inconsistent numbers of samples: [16757, 67026]